In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, auc, roc_curve
import lightgbm as lgb


%matplotlib inline

In [2]:
data_defaults = pd.read_csv("../data/UCI_Credit_Card.csv")
data_fraud = pd.read_csv("../data/creditcard.csv")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    data_defaults.drop(['default.payment.next.month','ID'],axis=1),
    data_defaults['default.payment.next.month'] ,
    test_size=0.33, 
    random_state=42
)

In [5]:
y_train.value_counts(normalize=True)

0    0.777214
1    0.222786
Name: default.payment.next.month, dtype: float64

In [110]:
# Old stuff

### Buld the default model

In [6]:
import lightgbm as lgb

train = lgb.Dataset(X_train, y_train)
test = lgb.Dataset(X_test, y_test, reference=train)


In [84]:
params_std = {
    'num_leaves': 8,
    'n_estimators':150, 
     'n_jobs': 2,
     'learning_rate': 0.02,
    'metric' : ['binary_logloss'],
    "verbose":0,
    "objective":"binary"
   }

lgb_std = lgb.train(params=params_std,
                train_set=train,
                valid_sets=[train, test],
                valid_names=['train','test'],
                early_stopping_rounds=10,
                verbose_eval=0,
               )


/Users/sandro/.miniconda/installation/envs/py38/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002279 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


In [85]:
roc_auc_score(y_test, lgb_std.predict(X_test))

0.7784170284061349

In [86]:
# lgb_std.trees_to_dataframe().head(20)

In [88]:
scale_pos_weight = y_train.value_counts().iloc[0]/y_train.value_counts().iloc[1]
print(scale_pos_weight)

params_balanced = params_std.copy()
params_balanced['scale_pos_weight']=2

lgb_balanced = lgb.train(params=params_balanced,
                train_set=train,
                valid_sets=[train, test],
                valid_names=['train','test'],
                early_stopping_rounds=10,
                verbose_eval=0
               )



3.4886109870477893
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001799 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


In [97]:
# lgb_balanced.predict?

In [99]:
std_preds = pd.DataFrame(
    lgb_std.predict(X_test, raw_score=True), index = X_test.index)
bal_preds =  pd.DataFrame(
    lgb_balanced.predict(X_test, raw_score=True),index = X_test.index
)

In [94]:
# df_trees = lgb_std.trees_to_dataframe()
# df_trees[(df_trees['tree_index']==1) & (df_trees['split_feature'].isnull())]

In [93]:
# df_trees = lgb_balanced.trees_to_dataframe()
# df_trees[(df_trees['tree_index']==1) & (df_trees['split_feature'].isnull())]

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
19,1,5,1-L0,None,None,1-S6,None,NaN,NaN,None,None,None,0.008811,993.190648,4849
20,1,5,1-L7,None,None,1-S6,None,NaN,NaN,None,None,None,0.035647,54.609206,227
21,1,4,1-L6,None,None,1-S5,None,NaN,NaN,None,None,None,0.045176,50.116138,197
23,1,4,1-L3,None,None,1-S3,None,NaN,NaN,None,None,None,-0.004515,1850.442470,9719
24,1,4,1-L4,None,None,1-S3,None,NaN,NaN,None,None,None,0.025302,122.174135,547
27,1,4,1-L1,None,None,1-S4,None,NaN,NaN,None,None,None,0.020032,271.109526,1237
28,1,4,1-L5,None,None,1-S4,None,NaN,NaN,None,None,None,0.042170,306.330492,1205
29,1,3,1-L2,None,None,1-S1,None,NaN,NaN,None,None,None,0.065214,646.256785,2119


In [78]:
std_preds[0].value_counts()

6    4368
3    1559
5    1227
7     953
0     848
2     755
4     132
1      58
Name: 0, dtype: int64

In [109]:
round((std_preds[0]/bal_preds[0]),2).sort_values()

2751    -527.54
1109    -314.56
9798    -302.41
14576   -164.35
7376    -157.47
          ...  
8735      98.73
20921    125.03
19025    228.07
7751     231.53
15443    866.20
Name: 0, Length: 9900, dtype: float64

In [ ]:
lgb_balanced.predict(X_train

In [55]:
lgb.sklearn.train?

Signature:
lgb.sklearn.train(
    params,
    train_set,
    num_boost_round=100,
    valid_sets=None,
    valid_names=None,
    fobj=None,
    feval=None,
    init_model=None,
    feature_name='auto',
    categorical_feature='auto',
    early_stopping_rounds=None,
    evals_result=None,
    verbose_eval=True,
    learning_rates=None,
    keep_training_booster=False,
    callbacks=None,
)
Docstring:
Perform the training with given parameters.

Parameters
----------
params : dict
    Parameters for training.
train_set : Dataset
    Data to be trained on.
num_boost_round : int, optional (default=100)
    Number of boosting iterations.
valid_sets : list of Datasets or None, optional (default=None)
    List of data to be evaluated on during training.
valid_names : list of strings or None, optional (default=None)
    Names of ``valid_sets``.
fobj : callable or None, optional (default=None)
    Customized objective function.
    Should accept two parameters: preds, train_data,
    and return

In [52]:
lgb_balanced.eval(train, name='train')

AttributeError: 'Booster' object has no attribute 'train_set'

In [44]:
roc_auc_score(y_test, lgb_balanced.predict(X_test))

0.7661501878587218

In [ ]:
params_balanced = params_std.copy()
params_balanced['class_weight']="balanced"

lgb_balanced = lgb.train(params=params_balanced,
                train_set=train,
                valid_sets=[train, test],
                valid_names=['train','test'],
#                 fobj=WeightedFocalLoss(alpha=alpha, gamma=gamma),
#                 feval=WeightedFocalMetric(alpha=alpha, gamma=gamma),
                early_stopping_rounds=10,
                verbose_eval=15
               )

